# Tutorial, chapter 6

- Generate vtm baseline results

In [20]:
# https://nbconvert.readthedocs.io/en/latest/removing_cells.html
# use these magic spells to update your classes methods on-the-fly as you edit them:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
# import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))
from PIL import Image
from matplotlib import pyplot as plt

/tmp/ipykernel_9490/416472496.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Markdown


In this chapter you will learn:

- to generate and cache VTM-encoded bitstream
- to create VTM baseline results

In [8]:
path_to_examples="/home/sampsa/silo/interdigital/CompressAI-Vision/examples"
path_to_vtm_software="/home/sampsa/silo/interdigital/VVCSoftware_VTM"

The subcommand ``vtm`` encodes images from your dataset with the VTM program.  VTM features state-of-the-art classical video encoding techniques and it is used as a benchmark against your deep-learning encoder's efficiency.  You need to download and compile the VTM software yourself according to the instructions in the main documentation.

Why do we need a separate subcommand ``vtm`` for VTM encoding / bitstream generation, instead of just using ``detectron2-eval`` on the fly? i.e. for doing:
```
Image --> VTM encoding --> VTM decoding --> Detectron2
```
That is because encoding performed by VTM is *very* CPU intensive task, so it is something you don't really want to repeat (encoding 5000 sample images, depending on your qpars value, can take several days..!), so we use the ``vtm`` subcommand to manage, encode and cache the VTM produced bitstreams on disk.

Let's generate some encoded bitstreams.

In [13]:
!rm -rf /tmp/bitstreams

In [14]:
!compressai-vision vtm --y --dataset-name=mpeg-vcm-detection \
--slice=0:2 \
--scale=100 \
--progress=1 \
--qpars=47 \
--vtm_cache=/tmp/bitstreams \
--vtm_dir={path_to_vtm_software}/bin \
--vtm_cfg={path_to_vtm_software}/cfg/encoder_intra_vtm.cfg \
--output=vtm_out.json

importing fiftyone
fiftyone imported
reading VTM config from '/home/sampsa/silo/interdigital/VVCSoftware_VTM/cfg/encoder_intra_vtm.cfg'

VTM bitstream generation
Target dir             : /tmp/bitstreams
Quality points/subdirs : [47]
Using dataset          : mpeg-vcm-detection
Image Scaling          : 100
Using slice            : 0:2
Number of samples      : 2
Progressbar            : False
Output file            : vtm_out.json
Print progress         : 1

QUALITY PARAMETER 47
VTMEncoderDecoder - INFO - creating /tmp/bitstreams
VTMEncoderDecoder - WARNING - creating bitstream /tmp/bitstreams/100/47/bin_0001eeaf4aed83f9 with VTMEncode from scratch
sample:  1 / 2 tag: 0001eeaf4aed83f9
VTMEncoderDecoder - WARNING - creating bitstream /tmp/bitstreams/100/47/bin_000a1249af2bc5f0 with VTMEncode from scratch
sample:  2 / 2 tag: 000a1249af2bc5f0

HAVE A NICE DAY!



As you can see, bitstreams we're generated and cached into ``/tmp/bitstreams/SCALE/QP``.  Let's see what happens if we run the exact same command again: 

In [15]:
!compressai-vision vtm --y --dataset-name=mpeg-vcm-detection \
--slice=0:2 \
--scale=100 \
--progress=1 \
--qpars=47 \
--vtm_cache=/tmp/bitstreams \
--vtm_dir={path_to_vtm_software}/bin \
--vtm_cfg={path_to_vtm_software}/cfg/encoder_intra_vtm.cfg \
--output=vtm_out.json

importing fiftyone
fiftyone imported
reading VTM config from '/home/sampsa/silo/interdigital/VVCSoftware_VTM/cfg/encoder_intra_vtm.cfg'

VTM bitstream generation
Target dir             : /tmp/bitstreams
Quality points/subdirs : [47]
Using dataset          : mpeg-vcm-detection
Image Scaling          : 100
Using slice            : 0:2
Number of samples      : 2
Progressbar            : False
Output file            : vtm_out.json
Print progress         : 1

QUALITY PARAMETER 47
VTMEncoderDecoder - WARNING - folder /tmp/bitstreams/100/47 exists already
sample:  1 / 2 tag: 0001eeaf4aed83f9
sample:  2 / 2 tag: 000a1249af2bc5f0

HAVE A NICE DAY!



Instead of generating the bitstreams, the program found them cached on the disk and just verified them.

Let's fool around and corrupt one of the bitstreams:

In [16]:
!echo " " > /tmp/bitstreams/100/47/bin_000a1249af2bc5f0

And run the command again:

In [18]:
!compressai-vision vtm --y --dataset-name=mpeg-vcm-detection \
--slice=0:2 \
--scale=100 \
--progress=1 \
--qpars=47 \
--vtm_cache=/tmp/bitstreams \
--vtm_dir={path_to_vtm_software}/bin \
--vtm_cfg={path_to_vtm_software}/cfg/encoder_intra_vtm.cfg \
--output=vtm_out.json

importing fiftyone
fiftyone imported
reading VTM config from '/home/sampsa/silo/interdigital/VVCSoftware_VTM/cfg/encoder_intra_vtm.cfg'

VTM bitstream generation
Target dir             : /tmp/bitstreams
Quality points/subdirs : [47]
Using dataset          : mpeg-vcm-detection
Image Scaling          : 100
Using slice            : 0:2
Number of samples      : 2
Progressbar            : False
Output file            : vtm_out.json
Print progress         : 1

QUALITY PARAMETER 47
VTMEncoderDecoder - WARNING - folder /tmp/bitstreams/100/47 exists already
sample:  1 / 2 tag: 0001eeaf4aed83f9
VTMEncoderDecoder - CRITICAL - VTM encode failed with Warning: Attempt to decode an empty NAL unit

VTMEncoderDecoder - CRITICAL - VTMDecode failed: will skip image 000a1249af2bc5f0 & remove the bitstream file
ERROR: Corrupt data for image id=633720fcee3965dd257f247c, tag=000a1249af2bc5f0, path=/home/sampsa/fiftyone/mpeg-vcm-detection/data/000a1249af2bc5f0.jpg
ERROR: Trying to regenerate
VTMEncoderDecoder

You can run the ``vtm`` command parallelized over *both* quality parameters *and* dataset slices in order to speed things up.  In the case of crashes / data corruption, you can just send the same scripts into your queue system over and over again if necessary.

Finally, you can run ``detectron2-eval`` for the VTM case like this:

In [19]:
!compressai-vision detectron2-eval --y --dataset-name=mpeg-vcm-detection \
--slice=0:2 \
--scale=100 \
--progress=1 \
--qpars=47 \
--vtm \
--vtm_cache=/tmp/bitstreams \
--vtm_dir={path_to_vtm_software}/bin \
--vtm_cfg={path_to_vtm_software}/cfg/encoder_intra_vtm.cfg \
--output=detectron2_vtm.json \
--model=COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml

importing fiftyone
fiftyone imported
Reading vtm config from: /home/sampsa/silo/interdigital/VVCSoftware_VTM/cfg/encoder_intra_vtm.cfg

Using dataset          : mpeg-vcm-detection
Dataset tmp clone      : detectron-run-sampsa-mpeg-vcm-detection-2022-10-11-18-21-25-081407
Image scaling          : 100
Number of samples      : 2
Torch device           : cpu
Detectron2 model       : COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml
Model was trained with : coco_2017_train
Using VTM               
Quality parameters     : [47]
Ground truth data field name
                       : detections
Eval. results will be saved to datafield
                       : detectron-predictions
Evaluation protocol    : open-images
Progressbar            : False
Print progress         : 1
Output file            : detectron2_vtm.json
Peek model classes     :
['airplane', 'apple', 'backpack', 'banana', 'baseball bat'] ...
Peek dataset classes   :
['airplane', 'person'] ...
cloning dataset mpeg-vcm-detection to